In [3]:
import pandas as pd
import numpy as np
import altair as alt

# Set the notebook to display all columns of a dataframe
pd.set_option('display.max_columns', None)

# import service now data 
inc_df = pd.read_csv(r'do_not_commit\Datasets\SerivceNow_Incident.csv')


C:\Users\JRankin\AppData\Local\Temp\ipykernel_4920\3670244283.py:9: DtypeWarning: Columns (0,2,3,4,8,23,37,42,47,48,49,57,67,74,80,86,91,92,94,97,101,103) have mixed types. Specify dtype option on import or set low_memory=False.
  inc_df = pd.read_csv(r'do_not_commit\Datasets\SerivceNow_Incident.csv')


In [8]:
# Reformat date time to date 
inc_df['opened_at'] = pd.to_datetime(inc_df['opened_at'])
inc_df['opened_at_formatted'] = inc_df['opened_at'].dt.strftime('%Y-%m-%d')

In [10]:
grouped_counts = inc_df.groupby(['opened_at_formatted', 'configuration_item']).size()
print(grouped_counts[grouped_counts > 1].sum())
grouped_counts[grouped_counts > 1]

416


opened_at_formatted  configuration_item
2023-01-03           CHI-L-U33511          2
                     LOE-D-U25294          2
                     MWE-L-U29955          2
2023-01-04           MIL-L-U21470          2
                     MIL-L-U30749          2
                                          ..
2023-07-05           MIL-L-U34247          2
                     TCF-L-U34494          2
2023-07-06           MIL-D-U25226          2
2023-07-10           CHI-L-U34590          2
                     PIT-L-U33237          2
Length: 200, dtype: int64